In [7]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from imblearn.over_sampling import *
from imblearn.under_sampling import RandomUnderSampler


In [8]:

# 준비된 수술 환자 데이터를 불러들입니다.
raw_df = pd.read_csv("sample_data/ThoraricSurgery.csv", header=None)
data_vale=raw_df.values

# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장합니다.
X = data_vale[:,0:17]
Y = data_vale[:,17:18]




In [9]:

# 전체 데이터에서 학습 데이터와 테스트 데이터(0.2)로 구분
X_train1, X_test, Y_train1, Y_test = train_test_split(X, Y, test_size=0.3)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train1, Y_train1, test_size=0.2)


In [10]:

X_samp, y_samp = RandomOverSampler(sampling_strategy=0.7).fit_sample(X_train, Y_train)
X_under_samp, y_under_samp =RandomUnderSampler(sampling_strategy=0.7).fit_sample(X_train, Y_train)
print("train class bin : " , np.bincount(Y_train.flatten().astype('int8')))
print("train OverSampler class bin : " , np.bincount(y_samp.flatten().astype('int8')))
print("train UnderSampler class bin : " , np.bincount(y_under_samp.flatten().astype('int8')))
print("val class bin : " , np.bincount(Y_valid.flatten().astype('int8')))
print("test class bin : " , np.bincount(Y_test.flatten().astype('int8')))
# https://datascienceschool.net/03%20machine%20learning/14.02%20%EB%B9%84%EB%8C%80%EC%B9%AD%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%AC%B8%EC%A0%9C.html


train class bin :  [220  43]
train OverSampler class bin :  [220 154]
train UnderSampler class bin :  [61 43]
val class bin :  [59  7]
test class bin :  [121  20]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [11]:

# 딥러닝 구조를 결정합니다(모델을 설정하고 실행하는 부분입니다).
input_Layer = tf.keras.layers.Input(shape=(17,))
x = tf.keras.layers.Dense(20, activation='sigmoid')(input_Layer)
Out_Layer= tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 17)]              0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                360       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


In [12]:

# 딥러닝을 실행합니다.
loss=tf.keras.losses.binary_crossentropy
optimizer = tf.keras.optimizers.SGD(learning_rate=0.009)
metrics=tf.keras.metrics.binary_accuracy

model.compile(loss=loss, optimizer=optimizer, metrics=[metrics])
result = model.fit(X_train, Y_train, epochs=1000, batch_size=100, validation_data=(X_valid,Y_valid))
print(result.history.keys())


Epoch 1/1000
3/3 [==============================] - 1s 71ms/step - loss: 1.2604 - binary_accuracy: 0.1635 - val_loss: 1.1340 - val_binary_accuracy: 0.1061
Epoch 2/1000
3/3 [==============================] - 0s 10ms/step - loss: 1.0834 - binary_accuracy: 0.1635 - val_loss: 1.0157 - val_binary_accuracy: 0.0909
Epoch 3/1000
3/3 [==============================] - 0s 9ms/step - loss: 0.9711 - binary_accuracy: 0.1597 - val_loss: 0.9071 - val_binary_accuracy: 0.1212
Epoch 4/1000
3/3 [==============================] - 0s 9ms/step - loss: 0.8378 - binary_accuracy: 0.2167 - val_loss: 0.7151 - val_binary_accuracy: 0.2576
Epoch 5/1000
3/3 [==============================] - 0s 8ms/step - loss: 0.7063 - binary_accuracy: 0.2510 - val_loss: 0.6720 - val_binary_accuracy: 0.8788
Epoch 6/1000
3/3 [==============================] - 0s 8ms/step - loss: 0.6670 - binary_accuracy: 0.8327 - val_loss: 0.6361 - val_binary_accuracy: 0.8788
Epoch 7/1000
3/3 [==============================] - 0s 8ms/step - loss: 0.

In [ ]:

### history에서 loss와 val_loss의 key를 가지는 값들만 추출
loss = result.history['loss']
val_loss = result.history['val_loss']

### loss와 val_loss를 그래프화
epochs = range(1, len(loss) + 1)
plt.subplot(211)  ## 2x1 개의 그래프 중에 1번째
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

### history에서 binary_accuracy와 val_binary_accuracy key를 가지는 값들만 추출
rmse = result.history['binary_accuracy']
val_rmse = result.history['val_binary_accuracy']
epochs = range(1, len(rmse) + 1)
### binary_accuracy와 val_binary_accuracy key를 그래프화
plt.subplot(212)  ## 2x1 개의 그래프 중에 2번째
plt.plot(epochs, rmse, 'ro', label='Training binary_accuracy')
plt.plot(epochs, val_rmse, 'r', label='Validation binary_accuracy')
plt.title('Training and validation binary_accuracy')
plt.xlabel('Epochs')
plt.ylabel('binary_accuracy')
plt.legend()

# 결과를 출력합니다.
print("-----")
print(model.evaluate(X_test, Y_test))
print("\n Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

plt.show()